In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Using code from DEMO
class BostonHousingDataset:
    def __init__(self):
        self.url = "http://lib.stat.cmu.edu/datasets/boston"
        self.feature_names = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]

    def load_dataset(self):
        # Fetch data from URL
        raw_df = pd.read_csv(self.url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

        # Create the dictionary in sklearn format
        dataset = {
            'data': [],
            'target': [],
            'feature_names': self.feature_names,
            'DESCR': 'Boston House Prices dataset'
        }

        dataset['data'] = data
        dataset['target'] = target

        return dataset

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3117646910.py:9: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(self.url, sep="\s+", skiprows=22, header=None)


In [ ]:
# Load dataset
boston_housing = BostonHousingDataset()
boston_dataset = boston_housing.load_dataset()

# Create dataset
df = pd.DataFrame(boston_dataset['data'], columns=boston_dataset['feature_names'])
df['MEDV'] = boston_dataset['target']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
print(df.isnull().sum())

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64


In [ ]:
# Split training and test sets
X = df.to_numpy()
X = np.delete(X, 13, axis=1)
y = df['MEDV'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

# scale the features so that gradients dont explode
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X.shape)

(506, 13)


In [ ]:
class LinearRegression:
  def __init__(self, learning_rate, iterations, loss_type):
    self.lr = learning_rate
    self.itr = iterations
    self.loss_type = loss_type
    self.b = 0 # Bias

  def fit(self, X, y):
    self.m, self.n = X.shape # rows(samples), cols(features)
    self.w = np.zeros(self.n) # initialize weights for each feature
    self.X = X
    self.y = y # true label

    # apply gradient descent for i iterations
    for i in range(self.itr):
      self.gradientDescent()
    return self

  def gradientDescent(self): # update parameters
    y_pred = self.predict(self.X)
    if self.loss_type == 'MSE':
      # calculate gradients (MSE)
      dw = (2/self.m)*np.dot(self.X.T, (y_pred - self.y))
      db = (2/self.m)*np.sum(y_pred - self.y)

    elif self.loss_type == 'MAE':
      # calculate gradients (MAE)
      dw = (1/self.m)*np.dot(self.X.T, np.sign(y_pred - self.y))
      db = (1/self.m)*np.sum(np.sign(y_pred - self.y))

    # gradient descent
    self.w = self.w - self.lr*dw
    self.b = self.b - self.lr*db
    return self

  def mean_squared_error(self, y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

  def mean_absolute_error(self, y_pred, y_true):
    return np.mean(np.abs(y_pred - y_true))

  def predict(self, X): # linear model
    return np.dot(X, self.w) + self.b


In [ ]:
# My mse and mae
model_mse = LinearRegression(learning_rate=0.01, iterations=10000, loss_type='MSE')
model_mse.fit(X_train, y_train)
mse_predictions = model_mse.predict(X_test)
test_mse = model_mse.mean_squared_error(mse_predictions, y_test)
train_mse_preds = model_mse.predict(X_train)
train_mse = model_mse.mean_squared_error(train_mse_preds, y_train)

print(f"My Test MSE: {test_mse}")
print(f"My Train MSE: {train_mse}")

model_mae = LinearRegression(learning_rate=0.01, iterations=10000, loss_type='MAE')
model_mae.fit(X_train, y_train)
mae_predictions = model_mae.predict(X_test)
test_mae = model_mae.mean_absolute_error(mae_predictions, y_test)
train_mae_preds = model_mae.predict(X_train)
train_mae = model_mae.mean_absolute_error(train_mae_preds, y_train)

print(f"My Test MAE: {test_mae}")
print(f"My Train MAE: {train_mae}")

My Test MSE: 20.86928360855995
My Train MSE: 22.47709040844014
My Test MAE: 3.0460680340404487
My Train MAE: 3.1090505893060665


In [ ]:
from sklearn.linear_model import LinearRegression as SKLinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
sk_model_mse = SKLinearRegression()
sk_model_mse.fit(X_train, y_train)
sk_mse_predictions = sk_model_mse.predict(X_test)
sk_mse = mean_squared_error(y_test, sk_mse_predictions)
print(f"SkLearn MSE: {sk_mse}")

sk_model_mae = SKLinearRegression()
sk_model_mae.fit(X_train, y_train)
sk_mae_predictions = sk_model_mae.predict(X_test)
sk_mae = mean_absolute_error(y_test, sk_mae_predictions)
print(f"SkLearn MAE: {sk_mae}")

SkLearn MSE: 20.869292183770813
SkLearn MAE: 3.2132704958423823


When I set my iterations to 10000, My linear regression model achieved an MSE of 20.66, which is close to SkLearn's MSE of 20.87. This shows that my gradient descent implementation for MSE is correctly converging to an optimal solution.

My MAE is 3.04 while SkLearn's MAE is 3.21. This shows that my implementation of MAE is also correctly converging to an optimal solution.

I found that when I set my iterations to 1000, My MAE has a noticeable change from 3.04 to 12.38. This is likely due to the convergence speed of MAE is constant; it doesn't slow down like MSE.

MSE: (y_pred - y)
MAE: np.sign(y_pred - y)
